In [1]:
# do not touch this cell 
%load_ext jupyter_record


# Set-up

Make sure that you have completed the set-up instuctions in Assignment 0. 

Every time you create a Jupyter cell in this assignment, it should be annotated with '%%git_commit' at the top. Please do not touch this line of code. If you do not see this line, make sure that you have moved the 'custom.js' file in the right location!


# Data Preprocessing

In this section, you should load your raw training dataset, and create your features for the model. You can find the data here: [shorturl.at/owMPS](https://shorturl.at/owMPS)

**IMPORTANT**: Download your data and place it in the "data" folder. You may run into errors if you place it anywhere else. You can use pandas to import your data from the csv files.

The data files are as following:

- training.csv: Seen/training data. Contains raw features and prediction class ('DepDel15').
- testing_X.csv: Unseen/testing data. Contains only raw features, and no prediction class. 
- readme.html: Information about the features. 



In [2]:
%%git_commit

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/training.csv')
print(df.head())

   Unnamed: 0  FlightDate            Airline Origin Dest  Cancelled  Diverted  \
0           0  2018-01-23  Endeavor Air Inc.    ABY  ATL      False     False   
1           1  2018-01-24  Endeavor Air Inc.    ABY  ATL      False     False   
2           2  2018-01-25  Endeavor Air Inc.    ABY  ATL      False     False   
3           3  2018-01-26  Endeavor Air Inc.    ABY  ATL      False     False   
4           4  2018-01-27  Endeavor Air Inc.    ABY  ATL      False     False   

   CRSDepTime  CRSElapsedTime  Distance  ...  DestAirportID  DestAirportSeqID  \
0        1202            62.0     145.0  ...          10397           1039707   
1        1202            62.0     145.0  ...          10397           1039707   
2        1202            62.0     145.0  ...          10397           1039707   
3        1202            62.0     145.0  ...          10397           1039707   
4        1400            60.0     145.0  ...          10397           1039707   

   DestCityMarketID  DestC

In [32]:
%%git_commit
Origin = pd.get_dummies(df['Origin'],drop_first = True)
Airline = pd.get_dummies(df['Airline'],drop_first = True)
DayOfWeek = pd.get_dummies(df['DayOfWeek'],drop_first = True)

new = pd.concat([Origin,Airline,DayOfWeek,df['DepDel15'],df['CRSDepTime'],df['CRSArrTime'],df['CRSElapsedTime']], axis = 1)
new.head()


,ABI,ABQ,ABR,ABY,ACK,ACT,ACV,ACY,ADK,ADQ,...,2,3,4,5,6,7,DepDel15,CRSDepTime,CRSArrTime,CRSElapsedTime
0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0.0,1202,1304,62.0
1,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0.0,1202,1304,62.0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0.0,1202,1304,62.0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0.0,1202,1304,62.0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0.0,1400,1500,60.0


In [20]:
%%git_commit
new2 = new.sample(n = 100000)
corr_check = (new2.corr()['DepDel15']).sort_values(ascending = False)
corr_check.dropna(inplace = True)
print(corr_check[corr_check > .05])

DepDel15                  1.000000
CRSDepTime                0.174535
CRSArrTime                0.137207
Southwest Airlines Co.    0.055447
Name: DepDel15, dtype: float64


In [21]:
%%git_commit
print(corr_check[abs(corr_check) > .03])

DepDel15                  1.000000
CRSDepTime                0.174535
CRSArrTime                0.137207
Southwest Airlines Co.    0.055447
JetBlue Airways           0.049576
Frontier Airlines Inc.    0.044479
MDW                       0.039009
Alaska Airlines Inc.     -0.030646
Hawaiian Airlines Inc.   -0.032304
Delta Air Lines Inc.     -0.048013
Name: DepDel15, dtype: float64


In [34]:
%%git_commit

bad = new['Southwest Airlines Co.'] | new['JetBlue Airways'] | new['Frontier Airlines Inc.'] | new['MDW']
good = new['Alaska Airlines Inc.'] | new['Hawaiian Airlines Inc.'] | new['Delta Air Lines Inc.']
time_dif = new['CRSArrTime'] - new['CRSDepTime'] - new['CRSElapsedTime']
from scipy.stats import pearsonr
print(df.shape)
pearsonr(time_dif,new['DepDel15'])


(5689512, 41)


ValueError: array must not contain infs or NaNs

In [17]:
%%git_commit
new.dropna(inplace = True)

# testing.dropna(inplace = True)
new2 = new.sample(n = 1000)
X = new2[['CRSDepTime','CRSArrTime','Southwest Airlines Co.','JetBlue Airways','Frontier Airlines Inc.','MDW','Alaska Airlines Inc.','Hawaiian Airlines Inc.','Delta Air Lines Inc.']].to_numpy()
Y = new2['DepDel15'].to_numpy()

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33)

model = SVC()
model.fit(X_train,Y_train)

Y_hat = model.predict(X_test)

In [19]:
%%git_commit
Y_hat.sum()-Y_test.sum()

0.0

In [ ]:
%%git_commit
print(df.columns)
filtered = df.select_dtypes(include='number') 
filtered.drop(columns = "Year",inplace = True)
# filtered.columns
filtered['Delay'] = filtered['CRSElapsedTime'] - (filtered['CRSArrTime'] - filtered['CRSArrTime'])
filtered.corr()

In [ ]:
%%git_commit
from sklearn.preprocessing import OneHotEncoder

ohe_airports = df['OriginAirportID'].to_numpy()
ohe = OneHotEncoder(sparse=False)

ohe_airports = ohe.fit_transform(ohe_airports.reshape(-1,1))
print(ohe_airports)
print(ohe_airports[0])

# Model Prediction

In this section, you should import relevant modeling/scikit-learn libraries and create a prediction model from your features. You should carefuly evaluate how your model performs based on the data you already have.

**Hint**: We are using a real world dataset with low correlation between the raw features and prediction class. You may have to carefully think about how to hand-engineer features from the raw features to capture correlation using a machine learning model.


**Hint 2**: When the instructors ran this assignment, we achieved an accuracy of 0.74 and AUC score of 0.54 on held out data. We do not expect high accuracy for this task, but your goal is to create a prediction model that is better than the simplest baseline - i.e. a model that predictions the majority class for all samples.


In [ ]:
%%git_commit
df.dropna(inplace = True)
result = df.select_dtypes(include='number')

print(result.head())
cr = result.corr()
cr = cr[abs(cr["DepDel15"]) > .02]
print(cr)

X = result[['CRSDepTime','CRSArrTime','OriginAirportID']].to_numpy()
Y = result['DepDel15'].to_numpy()

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
%%git_commit
print(X)

In [ ]:
%%git_commit
from sklearn.linear_model import LogisticRegression

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33)

print(len(X_train))
model = LogisticRegression()
model.fit(X_train,Y_train)

Y_hat = model.predict(X_train)


In [ ]:
%%git_commit
# (sum(Y_test) - sum(Y_hat))/len(Y_test[Y_test == 1])
print(len(Y_train[Y_train == 0]))
len(Y_train[Y_train == 1])/(len(Y_train[Y_train == 1]) + len(Y_train[Y_train == 0]))


# Final Evaluation

In this section, you should use your prediction model on held out data. For each sample in 'testing_X.csv', generate a binary prediction on whether the flight will be delayed by 15min or more.

**Important**: Create a csv file named that is a list of your predictions. It should be named './data/[CNET_ID]_final_assignment.csv' file, and contain only one row and columns equal to the sample size. 

**IMPORTANT**: Make sure that you create the new file in the "data" folder. You may run into errors if you place it anywhere else. 

Upload the '.csv' file here: [shorturl.at/aox05](https://shorturl.at/aox05)

This is your submission for the project! A part of your assessment will be on performance on this unseen dataset, and whether you can do better than majority class prediction. 

Note that you do not have true classification labels on this dataset.


In [ ]:
%%git_commit
df2 = pd.read_csv('data/testing_X.csv')
df2.head()


In [ ]:
%%git_commit
